# Tratamento dos arquivos de treinamento e teste

Nesse notebook trataremos os arquivos `trasact_train.csv`, `transact_class.csv` e `realclass_t1.csv`.

O objetivo é preparar os dados na forma de CSV para utiliza-los na fase seguinte, escolha de paramêtros dos algoritmo de classificação.

Ao final do processo serão gerados 3 pares de arquivos, cada par referente a uma estratégia para tratamento de missing values.

### Importar as bibliotecas necessárias

In [1]:
import pandas as pd
import numpy as np

### Ler os arquivos csv

In [ ]:
transact_train = pd.read_csv('transact_train.csv', sep="|", na_values='?')

transact_train.head()

In [ ]:
ant = data['sessionNo'][0]
indexes = []
training_data = pd.DataFrame()
for index, row in data.iterrows():
    if row['sessionNo'] != ant:
        indexes.append(index)      
    ant = row['sessionNo']
for index in range(0, len(indexes)):
    indexes[index] -= 1
indexes.append(len(data) - 1)

In [ ]:
training_data = data.iloc[indexes].set_index('sessionNo')
training_data.head()

In [ ]:
for col in training_data:
    if training_data[col].dtype == 'object':
        training_data[col].fillna(value=training_data[col].mode().iloc[0], inplace=True)
        print("Object", col)
    else:
        training_data[col].fillna(value=training_data[col].mean(), inplace=True)
training_data

In [ ]:
training_data.to_csv('training_data_mean.csv')